In [1]:
#!pip install bertopic datasets accelerate bitsandbytes xformers adjustText

# imports
import pandas as pd
import ipywidgets as widgets
import math
import os
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
import time
import en_core_web_sm


optimizer=Optimizer()
nlp = en_core_web_sm.load()

/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
#SETUP
DATA = "NL"
EM = "emanjavacas/GysBERT"
EM2 = "paraphrase-multilingual-MiniLM-L12-v2"
#jegormeister/bert-base-dutch-cased-snli
#paraphrase-multilingual-MiniLM-L12-v2

In [3]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(DATA)
docs = [" ".join(words) for words in dataset.get_corpus()]

In [4]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(EM2)
embeddings = embedding_model.encode(docs, show_progress_bar=True)


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [5]:
dataset, custom = DATA, True

In [6]:
from evaluation_vis import Trainer
import inspect

2024-06-10 20:24:26.864513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 20:24:27.082229: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# BERTopic

In [ ]:
#TC & TD CALCULATION
for i in range(3):
    custom = True
    params = {
        "embedding_model": embedding_model,
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 10,
        #"diversity": None,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                        model_name="BERTopic",
                        params=params,
                        bt_embeddings=embeddings,
                        custom_dataset=custom,
                        verbose=True)
    results = trainer.train(save=f"results/NL_mod/scores/bertopic_{i+1}")

In [ ]:
#TOPIC EXTRACTION
custom = True
params = {
    "embedding_model": embedding_model,
    "nr_topics": 15,
    "min_topic_size": 10,
    #"diversity": None,
    "verbose": True
}

trainer = Trainer(dataset=dataset,
                    model_name="BERTopic",
                    params=params,
                    bt_embeddings=embeddings,
                    custom_dataset=custom,
                    verbose=True)
results = trainer.train(save=f"results/NL_mod/topics/bertopic")

# LDA

In [ ]:
for i, random_state in enumerate([0, 21, 42]):
    dataset, custom = DATA, True
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": random_state}

    trainer = Trainer(dataset=dataset,
                      model_name="LDA",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"results/NL_mod/scores/lda_{i+1}")

In [ ]:
#TOPIC CREATION:
dataset, custom = DATA, True
params = {"num_topics": 15, "random_state": 42}#[(i+1)*10 for i in range(5)], "random_state": random_state}

trainer = Trainer(dataset=dataset,
                  model_name="LDA",
                  params=params,
                  custom_dataset=custom,
                  verbose=True)
results = trainer.train(save=f"results/NL_mod/topics/lda")

# NMF

In [ ]:
for i, random_state in enumerate([0, 21, 42]):
    dataset, custom = DATA, True
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": random_state}

    trainer = Trainer(dataset=dataset,
                      model_name="NMF",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"results/NL_mod/scores/nmf_{i+1}")

In [ ]:
#TOPIC CREATION:
dataset, custom = DATA, True
params = {"num_topics": 15, "random_state": 42}#[(i+1)*10 for i in range(5)], "random_state": random_state}

trainer = Trainer(dataset=dataset,
                  model_name="NMF",
                  params=params,
                  custom_dataset=custom,
                  verbose=True)
results = trainer.train(save=f"results/NL_mod/topics/nmf")

# CTM

In [ ]:
import nltk

#nltk.download("stopwords")
from nltk.corpus import stopwords

In [ ]:
# TC & TD
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
#!pip install contextualized_topic_models
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
for i in range(1):
    dataset, custom = DATA, True
    params = {
        "n_components": [(i+1)*10 for i in range(5)],
        "contextual_size":768
    }

    trainer = Trainer(dataset=dataset,
                      model_name="CTM_CUSTOM",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"results/NL_mod/scores/ctm_{i+1}")

In [ ]:
# TOPIC CREATION
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
#!pip install contextualized_topic_models
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
#for i in range(3):
dataset, custom = DATA, True
params = {
    "n_components": 15,#[(i+1)*10 for i in range(5)],
    "contextual_size":768
}

trainer = Trainer(dataset=dataset,
                  model_name="CTM_CUSTOM",
                  params=params,
                  custom_dataset=custom,
                  verbose=True)
results = trainer.train(save=f"results/NL_mod/topics/ctm")

# Dynamic TM

In [7]:
#SETUP
dDATA = "NL_dtm"

In [8]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(dDATA)

In [9]:
from sentence_transformers import SentenceTransformer
dataset, custom = dDATA, True
from data_t_NL import DataLoader
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
#print(timestamps)
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

embedding_model = SentenceTransformer(EM2) #EM3
embeddings = embedding_model.encode(docs, show_progress_bar=True)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [10]:
# Match indices
import os
os.listdir(f"./{dataset}")
with open(f"./{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
timestamps = [timestamp for index, timestamp in enumerate(timestamps) if index in indices]
len(data), len(timestamps)

(9997, 9997)

In [11]:
from evaluation import Trainer

params = {
        "nr_topics": 10,#[(i+1)*10 for i in range(5)],
        "min_topic_size": 5,
        "verbose": True,
    }

trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings,
                      custom_dataset=custom,
                      bt_timestamps=timestamps,
                      topk=5,
                      bt_nr_bins=10,
                      verbose=True)
results = trainer.train(f"NOT_NEEDED/D_NL_T_EVO_M")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-06-10 20:24:41,536 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-10 20:25:18,319 - BERTopic - Dimensionality - Completed ✓
2024-06-10 20:25:18,320 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-10 20:25:18,826 - BERTopic - Cluster - Completed ✓
2024-06-10 20:25:18,827 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-10 20:25:20,384 - BERTopic - Representation - Completed ✓
2024-06-10 20:25:20,385 - BERTopic - Topic reduction - Reducing number of topics
2024-06-10 20:25:21,714 - BERTopic - Topic reduction - Reduced number of topics from 242 to 10
10it [00:01,  5.39it/s]


esperana
harmonien
waanideen
efezirs
isral
gesoleerd
aron
baum
mystres
brger
normandi
creer
angoulme
gelimineerd
tween
arons
patinten
itali
azi
indivaarder
vogelvolire
sicili
gttlichen
hermsilaos
reen
caster
olin
zeen
frdrique
zweetporin
mattes
genstalleerd
coup
